# CS246 - Colab 9
## Studying COVID-19

In [1]:
from IPython.display import Image
print("Colab 9 Mascot saying Bye-Bye :( \n")
Image(url='https://media.giphy.com/media/tuvMgAPzxaQBq/giphy.gif',width=200)

Colab 9 Mascot saying Bye-Bye :( 



### Setup

Let's set up Spark on your Colab environment.  Run the cell below!

In [2]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=38763455ff8ae052bb80507ab52484a8a2f365b5d3fcb0a04823f1c7a6fcc060
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark
The following additional packages will be installed:
  libxtst6 openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra fonts-nanum fonts-ipafont-gothic
  fonts-ipafont-mincho fonts-wqy-microhei fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  libxtst6 openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 3 newly installed, 0 to remove and 39 not upgraded.
Need to get 39.7 MB of archives.
After this operation, 144 MB of additional disk space will be used.
Selecting previously unselected package

Now we authenticate a Google Drive client to download the files we will be processing in our Spark job.

**Make sure to follow the interactive instructions.**

In [3]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [4]:
id='1YT7ttUAafCjbVdm6obeHp1TWAK0rEtoR'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('time_series_covid19_confirmed_global.csv')

id='1YxEA5UQ2EFJ_9oLssM__Gs1ncVNufGNA'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('time_series_covid19_deaths_global.csv')

id='1CNxszuZTeIw-5cF5yrzKMZdb1qV0hSoy'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('time_series_covid19_recovered_global.csv')

If you executed the cells above, you should be able to see the dataset we will use for this Colab under the "Files" tab on the left panel.

Next, we import some of the common libraries needed for our task.

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import pyspark
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf

Let's initialize the Spark context.

In [6]:
# create the session
conf = SparkConf().set("spark.ui.port", "4050")

# create the context
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

### Data Loading

In this Colab, we will be analyzing the time series data of the Coronavirus COVID-19 Global Cases, collected by Johns Hopkins CSSE.

Here you can check a realtime dashboard based on this dataset: [https://www.arcgis.com/apps/opsdashboard/index.html?fbclid=IwAR2hQKsEZ3D38wVtXGryUhP9CG0Z6MYbUM_boPEaV8FBe71wUvDPc65ZG78#/bda7594740fd40299423467b48e9ecf6](https://www.arcgis.com/apps/opsdashboard/index.html?fbclid=IwAR2hQKsEZ3D38wVtXGryUhP9CG0Z6MYbUM_boPEaV8FBe71wUvDPc65ZG78#/bda7594740fd40299423467b48e9ecf6)

---



*   ```confirmed```: dataframe containing the cumulative number of confirmed COVID-19 cases, divided by geographical area
*   ```deaths```: dataframe containing the cumulative number of deaths due to COVID-19, divided by geographical area
*   ```recovered```: dataframe containing the cumulative number of recoevered patients, divided by geographical area

The data sets contain data entries for each day, representing the cumulative totals as of that day.







In [53]:
confirmed = spark.read.csv('time_series_covid19_confirmed_global.csv', header=True)
deaths = spark.read.csv('time_series_covid19_deaths_global.csv', header=True)
recovered = spark.read.csv('time_series_covid19_recovered_global.csv', header=True)

In [8]:
confirmed.show()

+--------------------+-------------------+---------+----------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+------+------+------+------+------+------+------+------+------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+------+------+------+------+------+------+------+------+------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+------+------+------+------+------+------+------+------+------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+------+------+------+------+------+------+------+------+------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-----

### Your Task

We are aware of the stress we are all experiencing because of the fact that many of you have projects and exams due this week. As such, we decided to conclude our series of Colabs with a **lightweight task** -- given everything you have learned about Spark during the quarter, this Colab should take you just a few minutes to complete.

Consider the entries for May 1, 2021, in the timeseries, and compute:


*   number of confirmed COVID-19 cases across the globe
*   number of deaths due to COVID-19 (across the globe)
*   number of recovered patients across the globe



In [15]:
import pyspark.sql.functions as F

print("Confirmed cases: ", confirmed.agg(sum("5/1/21")).collect()[0][0])
print("Deaths: ", deaths.agg(F.sum("5/1/21")).collect()[0][0])
print("Recovered cases: ", recovered.agg(F.sum("5/1/21")).collect()[0][0])

Confirmed cases:  305066666.0
Deaths:  3192930.0
Recovered cases:  88919401.0


Consider the data points for March 1, 2020, and March 1, 2021, and filter out the geographical locations where less than 50 cases have been confirmed.
For the areas still taken into consideration after the filtering step, compute the ratio between number of deaths and number of confirmed cases.

(Hint:
1. You don't need to sum over the country(combine Province/State having the same Country/Region). In other words, the column Province/State should exist in the final dataframe for this question.
2. For Q2.6, the computed death rate should be in the range [0.2, 0.3] and your reported country should be a country in the middle east.)

In [46]:
# March 1, 2020

deaths2020 = deaths.select("Province/State", "Country/Region", "3/1/20", "5/18/21")\
                   .withColumn("deaths", col("5/18/21") - col("3/1/20"))

confirmed.select("Province/State", "Country/Region", "3/1/20", "5/18/21")\
         .filter(confirmed["3/1/20"] >= 50)\
         .withColumn("confirmed", col("5/18/21") - col("3/1/20"))\
         .join(deaths2020, on=["Province/State", "Country/Region"], how="inner")\
         .withColumn("Ratio", col("deaths")/col("confirmed"))\
         .orderBy(col("Ratio").desc())\
         .show()

+--------------+--------------+------+-------+---------+------+-------+------+--------------------+
|Province/State|Country/Region|3/1/20|5/18/21|confirmed|3/1/20|5/18/21|deaths|               Ratio|
+--------------+--------------+------+-------+---------+------+-------+------+--------------------+
|         Hubei|         China| 66907|  68159|   1252.0|  2761|   4512|1751.0|   1.398562300319489|
|        Hainan|         China|   168|    188|     20.0|     5|      6|   1.0|                0.05|
|     Hong Kong|         China|    96|  11826|  11730.0|     2|    210| 208.0| 0.01773231031543052|
|      Shandong|         China|   758|    883|    125.0|     6|      7|   1.0|               0.008|
|       Shaanxi|         China|   245|    606|    361.0|     1|      3|   2.0| 0.00554016620498615|
|         Jilin|         China|    93|    573|    480.0|     1|      3|   2.0|0.004166666666666667|
|      Liaoning|         China|   122|    422|    300.0|     1|      2|   1.0|0.003333333333333...|


In [66]:
# March 1, 2021

deaths2021 = deaths.select("Province/State", "Country/Region", "3/1/21", "5/18/21")\
                      .withColumn("deaths", col("5/18/21") - col("3/1/21"))\
                      .groupBy("Country/Region")\
                      .agg(sum("deaths").alias("deaths"))

confirmed50 = confirmed.select("Province/State", "Country/Region", "3/1/21", "5/18/21")\
                       .withColumn("confirmed", col("5/18/21") - col("3/1/21"))\
                       .groupBy("Country/Region")\
                       .agg(sum("confirmed").alias("confirmed"))\
                       .filter(col("confirmed") >=50)\
                       .join(deaths2021, on=["Country/Region"], how="inner")\
                       .withColumn("Ratio", col("deaths")/col("confirmed"))\
                       .orderBy(col("Ratio").desc())\
                       .show()

+--------------------+---------+-------+--------------------+
|      Country/Region|confirmed| deaths|               Ratio|
+--------------------+---------+-------+--------------------+
|               Yemen|   4276.0|  663.0|  0.1550514499532273|
|               Sudan|   4525.0|  556.0| 0.12287292817679558|
|              Mexico| 296231.0|34594.0| 0.11678048549949195|
|             Lesotho|    295.0|   25.0|  0.0847457627118644|
|               Syria|   8188.0|  678.0| 0.08280410356619443|
|             Somalia|   7183.0|  519.0| 0.07225393289711819|
|            Slovakia|  78967.0| 4978.0|  0.0630389909709119|
| Antigua and Barbuda|    482.0|   28.0|0.058091286307053944|
|               Egypt|  65068.0| 3705.0| 0.05694043154853384|
|Bosnia and Herzeg...|  71696.0| 3999.0|  0.0557771702744923|
|        South Africa| 103881.0| 5263.0| 0.05066374024123757|
|              Malawi|   2223.0|  105.0| 0.04723346828609987|
|            Zimbabwe|   2480.0|  115.0|0.046370967741935484|
|       

Consider the data points for March 1, 2021, and May 1, 2021, in the timeseries, and filter out the geographical locations where less than 50 deaths have been confirmed (as of March 1, 2021).
For the areas still taken into consideration after the filtering step, compute **the percent increase in cumulative deaths**(not difference in death rate) between the two dates.

In [69]:
deaths_percent = deaths.select("Province/State", "Country/Region", "3/1/21", "5/1/21")\
                       .groupBy("Country/Region")\
                       .agg(sum("3/1/21").alias("3/1/21"), sum("5/1/21").alias("5/1/21"))\
                       .withColumn("Difference", col("5/1/21") - col("3/1/21"))\
                       .filter(col("Difference") >= 50)\
                       .withColumnRenamed("3/1/21", "March deaths")\
                       .withColumnRenamed("5/1/21", "May deaths")\
                       .withColumn("Percent increase", (col("May deaths") - col("March deaths")) / col("March deaths"))\
                       .orderBy(col("Percent increase").desc())\
                       .show()

# confirmed.select("Province/State", "Country/Region", "3/1/21", "5/1/21")\
#          .filter(confirmed["3/1/21"] < 50)\
#          .withColumnRenamed("3/1/21", "March confirmed")\
#          .withColumnRenamed("5/1/21", "May confirmed")\
#          .join(deaths_percent, on=["Province/State", "Country/Region"], how="inner")\
#          .withColumn("Percent increase", (col("March deaths") - col("May deaths")) / col("May deaths"))\
#          .orderBy(col("Percent increase").desc())\
#          .show()

+--------------------+------------+----------+----------+------------------+
|      Country/Region|March deaths|May deaths|Difference|  Percent increase|
+--------------------+------------+----------+----------+------------------+
|            Mongolia|         2.0|     115.0|     113.0|              56.5|
|    Papua New Guinea|        14.0|     115.0|     101.0| 7.214285714285714|
|             Uruguay|       611.0|    2669.0|    2058.0|3.3682487725040917|
|             Somalia|       243.0|     713.0|     470.0|1.9341563786008231|
|            Thailand|        83.0|     224.0|     141.0|1.6987951807228916|
|            Djibouti|        63.0|     145.0|      82.0|1.3015873015873016|
|          Madagascar|       297.0|     654.0|     357.0| 1.202020202020202|
|            Botswana|       332.0|     712.0|     380.0| 1.144578313253012|
|            Paraguay|      3198.0|    6476.0|    3278.0|1.0250156347717323|
|                Cuba|       324.0|     654.0|     330.0|1.0185185185185186|

Once you have working code for each cell above, **head over to Gradescope, read the questions carefully, and submit your solution for this Colab**!